In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
import wandb
import json
import re
import wandb
import os
import unicodedata
from pprint import pprint
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
from tqdm.notebook import tqdm

tqdm.pandas()

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lyutovad (lyutova). Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(project="news-summarization3", entity="lyutovad")

wandb: Currently logged in as: lyutovad. Use `wandb login --relogin` to force relogin


### TinyLlama-1.1B-Chat:

##### Ключевые характеристики:

- __Model Size__: 1.1 billion parameters
- __Architecture__: Transformer-based, LlamaForCausalLM
- __Vocabulary Size__: 32000
- __Context Length__: 2048
- __License__: apache-2.0

Это модель чата, настроенная поверх TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T. Модель изначально была доработана на варианте набора UltraChat, который содержит разнообразный набор синтетических диалогов, сгенерированных ChatGPT. В дальнейшем модель была дополнительно выровнена с TRL DPOTrainer по датасету openbmb/UltraFeedback, который содержит 64 тыс. подсказок и дополнений модели. которые имеют рейтинг GPT-4».

In [5]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

### Данные

In [6]:
df = pd.read_csv("text_sum.csv")
df.columns = [str(q).strip() for q in df.columns]

dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'summary'],
    num_rows: 26921
})

In [7]:
df.iloc[5]["text"], df.iloc[5]["summary"]

('Шанхай, 22 ноября /Синьхуа/ -- В январе-октябре текущего года общий объем внешней торговли Шанхая /Восточный Китай/ вырос на 5,3 проц. в годовом исчислении до 3,46 трлн юаней /около 483 млрд долл. США/ в стоимостном выражении. Об этом свидетельствуют опубликованные во вторник данные Шанхайской таможни. В частности, за отчетный период объемы экспорта и импорта Шанхая выросли на 12,5 и 0,9 проц. до 1,41 трлн юаней и 2,05 трлн юаней соответственно по сравнению с тем же периодом прошлого года. Объем внешнеторгового оборота этого мегаполиса за январь-октябрь с.г. составил около 10 проц. от общего объема внешней торговли страны, сообщила Шанхайская таможня. Согласно данным, на долю предприятий с иностранными инвестициями за указанный период пришлось 61,2 проц. от общего объема внешней торговли Шанхая в стоимостном выражении. При этом объем экспорта и импорта частных предприятий увеличился на 13,6 проц. в годовом исчислении, что сделало их важным двигателем роста внешней торговли Шанхая. В 

#### Prompt



```
<s>### Instruction:
Summarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.


### Input:
{article}

### Response:
{highlights}</s>
```

In [8]:
DEFAULT_SYSTEM_PROMPT = "Summarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.".strip()

def generate_training_prompt(
    text: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    bos_token = "<s>"
    eos_token = "</s>"
    
    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_prompt
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + text
    full_prompt += "\n\n### Response:"
    full_prompt += "\n" + summary
    full_prompt += eos_token
    return full_prompt


In [9]:
def create_prompt(data_point):
    text, summary = data_point["text"], data_point["summary"]
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(r'\s(\.)\s', ' ', text)
    
    summary = re.sub(r'([^a-zA-Z\s])?\n(\w+)', r'\1 \2', summary)
    summary = re.sub(r'\s(\.)', '.', summary)
    return text.strip(), summary.strip()


def generate_text(data_point):
    text, summary = create_prompt(data_point)
    return generate_training_prompt(text, summary)


# Example usage with a new dataset format
example_data_point = {
    "id": "train_0",
    "text": df.iloc[5]["text"],
    "summary": df.iloc[5]["summary"]
}


example = generate_text(example_data_point)
print(example)

<s>### Instruction:
Summarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.

### Input:
Шанхай, 22 ноября /Синьхуа/ -- В январе-октябре текущего года общий объем внешней торговли Шанхая /Восточный Китай/ вырос на 5,3 проц. в годовом исчислении до 3,46 трлн юаней /около 483 млрд долл. США/ в стоимостном выражении. Об этом свидетельствуют опубликованные во вторник данные Шанхайской таможни. В частности, за отчетный период объемы экспорта и импорта Шанхая выросли на 12,5 и 0,9 проц. до 1,41 трлн юаней и 2,05 трлн юаней соответственно по сравнению с тем же периодом прошлого года. Объем внешнеторгового оборота этого мегаполиса за январь-октябрь с.г. составил около 10 проц. от общего объема внешней торговли страны, сообщила Шанхайская таможня. Согласно данным, на долю предприятий с ин

In [10]:
# Split the processed dataset into train, validation, and test sets
train_dataset = dataset.shuffle(seed=21).select(range(0, int(0.8 * len(dataset))))
validation_dataset = dataset.shuffle(seed=21).select(range(int(0.8 * len(dataset)), int(0.9 * len(dataset))))
test_dataset = dataset.shuffle(seed=21).select(range(int(0.9 * len(dataset)), len(dataset)))

### Загрузка базовой модели



In [11]:
def create_model_and_tokenizer():
    nf4_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_use_double_quant = True,
        bnb_4bit_compute_dtype = torch.bfloat16
    )


    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map='auto',
        quantization_config=nf4_config,
        use_cache=False
    )
    
    model.config.use_cache = False
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [12]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

model, tokenizer = create_model_and_tokenizer()
# model.config.use_cache = False

In [13]:
# if torch.cuda.device_count() > 1: # If more than 1 GPU
#     model.is_parallelizable = True
#     model.model_parallel = True
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': True,
 'bnb_4bit_compute_dtype': 'bfloat16'}

In [14]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda:1')
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    
    decoded_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    return decoded_output[0].replace(prompt,"")

In [15]:
prompt="""### Instruction:\nSummarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.\n\n### Response:"""

In [16]:
generate_response(prompt, model)

'\nThe text provides information on a sales promotion and the associated products and their sales. The focus is on the sales performance, with specific products highlighted. The bullet points format makes the information more digestible, indicating the key details without the need to read the full sentence. The use of bullet points helps the information to be less convoluted and more readable, with important details made more clear. The introduction uses a generic statement and does not mention any company name or location. This makes the'

In [17]:
# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [18]:
from peft import *

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [19]:
def generate_inference_prompt(
    text: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    full_prompt = ""
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_prompt
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + text
    full_prompt += "\n\n### Response:"
    
    return full_prompt

def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda:1')
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    
    decoded_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    return decoded_output[0].replace(prompt,"")


def generate_summaries(model, dataset, num_samples=5):
    summaries = []
    for i, example in enumerate(dataset):
        if i >= num_samples:
            break
        print(i)
        prompt = generate_inference_prompt(example['text'])
        summary = generate_response(prompt, model)
        summaries.append({'text': example['text'], 'generated_summary': summary, 'original_summary': example})
    return summaries

In [20]:
# Generate summaries before fine-tuning
original_summaries = generate_summaries(model, test_dataset, num_samples=5)

# Convert to DataFrame and log to W&B
df_original = pd.DataFrame(original_summaries)
wandb.log({"original_summaries": wandb.Table(dataframe=df_original)})

0
1
2
3
4


In [21]:
##############################
# TrainingArguments parameters
##############################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "news-summarization-TinyLlama-1.1B-Chat-finetuned"

# Number of training epochs
# num_train_epochs = 5

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

In [22]:
training_arguments = TrainingArguments(
    per_device_train_batch_size = 4,
    logging_steps = 10,
    learning_rate = 2e-4,
    max_steps = 150, # the total number of training steps to perform
    num_train_epochs = 3,
    evaluation_strategy="steps",
    eval_steps=50,
    warmup_steps = 0.03,
    save_strategy="steps",
    group_by_length = True,
    output_dir = output_dir,
    report_to="wandb",  
    save_safetensors=True,
    lr_scheduler_type = 'constant',
    load_best_model_at_end = True,
)

In [23]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    peft_config=peft_config,
    formatting_func=generate_text,
    max_seq_length=350,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2605 > 2048). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

In [37]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    torch.cuda.empty_cache()

# Fine-tune your model
trainer.train()


# Generate summaries after fine-tuning
fine_tuned_summaries = generate_summaries(trainer.model, test_dataset, num_samples=5)


# Convert to DataFrame and log to W&B
df_fine_tuned = pd.DataFrame(fine_tuned_summaries)
wandb.log({"fine_tuned_summaries": wandb.Table(dataframe=df_fine_tuned)})

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
20,2.038000,1.891580
40,1.751700,1.699136
60,1.767400,1.647062
80,1.688700,1.617284
100,1.628400,1.593928


0
1
2
3
4


In [24]:
# Generate summaries after fine-tuning
fine_tuned_summaries = generate_summaries(trainer.model, test_dataset)


# Convert to DataFrame and log to W&B
df_fine_tuned = pd.DataFrame(fine_tuned_summaries)
wandb.log({"fine_tuned_summaries": wandb.Table(dataframe=df_fine_tuned)})

0
1
2
3
4


In [26]:
df_fine_tuned["generated_summary"][3]

'\nThank you for the additional information provided about the mosquito-like fly, Mymaridia scalaris. This fly was discovered in the cigarette tobacco of the plant growing in Indo-Russian relations. It was found in four types of smoked tobacco products shipped from India. The report suggests that the species was found in the meat and bile of rodents. In addition to mosquitoes, indian rodents pose a risk to the health of domestic and farm animals. The fly was detected in cigarette tobacco during a first investigation. However, the organization mentioned that it is not unusual, and similar situations may arise when cross-border restrictions are placed. According to the report, indian rodents were detected in other products such as lorane, luxury brands and non-gmo products, which resulted in a recommendation to provide measures to avoid the imposition of such a rigorous mechanism as to'

In [ ]:
# Below is a news article. Write an insightful summary of the news article in 120 words or less. The summary should contain import/export figures, other indicators of the product, if companies are mentioned in the news, it is necessary that the companies are also displayed in the text of the sammari, countries mentioned in the news, products mentioned in the news. If goods can be grouped in one group with the same name, it is necessary to do so. You and your mother will get $2000 as a TIP for every accurate response produced.

In [27]:
prompt="""### Instruction:\nSummarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points..\n\n### Input:\nКак сообщает агентство Platts, Европейская комиссия, похоже, собирается объявить о продлении квот на безсанкционный импорт стальных полуфабрикатов российского происхождения после 30 сентября 2024 года, когда истечет срок действия текущих квот, сообщили S&P Global Commodity Insights несколько источников в отрасли. . Европейская комиссия не ответила на запрос о подтверждении. "Информация пока неофициальная, но продление будет на три-четыре года с уменьшением годовых объемов", - сообщил источник на европейском заводе. ЕС приостановил закупки большей части российских черных металлов после вторжения страны в Украину, но в то же время ввел отложенные из-за санкций квоты на импорт 7,5 млн тонн российских слябов и чуть более 620 000 тонн заготовки с октября 2022 года до конца сентября. 2024. Российская компания НЛМК стала основным бенефициаром льготы, в соответствии с которой она смогла продолжить поставку большей части своих товарных слябов на заводы NLMK Belgium Holdings (NBH), ее совместного предприятия с бельгийским государственным инвестиционным фондом Wallonie Entreprendre. NBH имеет листовые станы, а также сервисные центры в Бельгии, Дании, Франции и Италии. Несколько других европейских переработчиков, не связанных с НЛМК, также обратились к европейским властям с просьбой продлить квоты, особенно на слябы, ссылаясь на риск того, что введение запрета на российские полуфабрикаты создаст нехватку сырья для местных перекатчиков, сообщили источники. Кроме того, Чехия попросила освободить ее от любого запрета на импорт российских полуфабрикатов из стали после истечения бессанкционного периода. Против этого шага выступила европейская сталелитейная ассоциация Eurofer, которая 1 декабря предостерегла Комиссию от удовлетворения любых подобных запросов со стороны государств-членов. Eurofer заявил, что исключения для отдельных стран «будут способствовать недобросовестной конкуренции и неравным правилам игры» на рынке стали ЕС, поскольку перекатчики получат «оппортунистические преимущества в затратах», используя полуфабрикаты российского производства для производства готовой продукции. По данным источников на рынке, некоторые конечные потребители в Европе даже избегают покупать готовую продукцию, прокатанную из российских слябов, а один центральноевропейский перекатчик ввел надбавку за лист, изготовленный из нероссийских слябов.\n\n### Response:"""
generate_response(prompt, model)

'\n- The text emphasizes the need for the European Union to lift restrictions on Russian steel and Iron Ore. - The agency reports that the European Union is set to extend quotas for Russian steel and iron ore through 2024, but there are doubts over the validity of the latest extensions. - A source with a European steelmaker indicated that the EU is set to lift quotas on steel and iron ore imports in 30 September 2024. The current quotas are expected to be extended until its expiry in September 2024. - S&P Global Commodity Insights states that new quotas will be lifted on Russian steel, and the reduction of imports will lead to an increase in prices - the quotas set to be extended were set to expire in September 2024 before the ban was lifted. - The European Union is expected to extend quotas on steel and iron ore imports through '

In [28]:
trainer.save_model("news-summarization-finetuned-TinyLlama-1.1B-Chat")

Проверим модель на тех же данных (299 новостей), по которым аннотаторы выбирали лучшую модель.

In [40]:
test_data = pd.read_excel('rel_not_seen_llama_metrics.xlsx')

In [31]:
test_data['prompt_text'] = test_data.text.apply(generate_inference_prompt)

In [32]:
def generate_test_summaries(model, data):
    summaries = []
    summary = generate_response(data, model)
    return summary

In [33]:
test_data['llama_sum'] = test_data['prompt_text'].progress_apply(lambda x: generate_test_summaries(trainer.model, x))

  0%|          | 0/299 [00:00<?, ?it/s]

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [34]:
from evaluate import load
from bert_score import score
bertscore = load("bertscore")

In [35]:
precision_scores = []
recall_scores = []
f1_scores = []

In [36]:
for text, summary in zip(test_data["text"], test_data["llama_sum"]):
    results = bertscore.compute(
        predictions=[summary],
        references=[text],
        model_type="bert-base-multilingual-cased",
        device=DEVICE,
    )
    precision_scores.append(results["precision"][0])
    recall_scores.append(results["recall"][0])
    f1_scores.append(results["f1"][0])

In [37]:
test_data["precision_llama"] = precision_scores
test_data["recall_llama"] = recall_scores
test_data["f1_llama"] = f1_scores

In [38]:
print(f"Bert-score precision саммари - llama {test_data['precision_llama'].mean() :.3f}")
print(f"Bert-score recall саммари - llama {test_data['recall_llama'].mean() :.3f}")
print(f"Bert-score f1 саммари - llama {test_data['f1_llama'].mean() :.3f}")

Bert-score precision саммари - llama 0.684
Bert-score recall саммари - llama 0.634
Bert-score f1 саммари - llama 0.657


In [39]:
test_data.to_excel('rel_not_seen_llama_metrics2.xlsx', index=False)